In [37]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [38]:
# Load the data files
area_names = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\auxillary\\area_names.csv")
property_designs = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\auxillary\\property_designs.csv")
property_types = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\auxillary\\property_types.csv")
data = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\processed\\CLEAN_RoomPrice_11_06_2024.csv")

## There will be several feature engineering

1. Date Features: Convert month numeric to string, Holiday Indicator, Convert to Datetime, Stay Duration, Booking Month, Day of Week, Booking Lead Time
2. Revenue Metrics: Total Earnings, Price Fluctuation, 
3. Review Metrics: Total Review Score
4. Data Cleaning: Drop Unnecessary Columns

In [32]:
def feature_engineering(df, date_column, check_out_column, holiday_ranges):
    # Ensure the date columns are datetime objects
    df[date_column] = pd.to_datetime(df[date_column])
    df[check_out_column] = pd.to_datetime(df[check_out_column])
    
    # Extract the month from the date column and convert it to a categorical variable
    df['month'] = df[date_column].dt.strftime('%B').str.lower()

    # Initialize the 'contain_national_holiday' column with False
    df['contain_national_holiday'] = False
    
    # Check if the dates fall within any of the holiday ranges
    for start_date, end_date in holiday_ranges:
        mask = (df[date_column] <= end_date) & (df[check_out_column] >= start_date)
        df.loc[mask, 'contain_national_holiday'] = True
    
    return df

### Define holiday ranges (example range including Indonesia's Independence Day)

In [33]:
holiday_ranges = [(pd.to_datetime('2024-08-11'), pd.to_datetime('2024-08-21'))]

## Apply featture

In [34]:
df = feature_engineering(data, 'booking_check_in', 'booking_check_out', holiday_ranges)
df.head()


,Unnamed: 0,room_id,unit_id,booking_id,booking_check_in,created_at,booking_check_out,earnings_in_idr,earnings_per_day,average_baseline_price,rating,review_sentiment_score,communication,cleanliness,accuracy,month,contain_national_holiday
0,13,397667.0,UN2501,07-05-2023MartaRudy,2023-05-07,2023-05-07 08:58,2023-05-08,748425.01,748425.01,6.357513e+05,4.610057,0.665004,4.695459,4.572559,4.672231,may,False
1,25,397667.0,UN2501,1085232310,2023-09-02,2023-08-24 19:44,2023-09-06,2801652.00,700413.00,6.357513e+05,4.610057,0.665004,4.695459,4.572559,4.672231,september,False
2,26,394914.0,UN0375,1085251539,2023-09-11,2023-08-25 02:12,2023-09-13,2475126.00,1237563.00,1.325331e+06,4.610057,0.665004,4.695459,4.572559,4.672231,september,False
3,29,387370.0,UN2760,1166180157,2024-01-20,2024-01-17 10:00,2024-01-21,1664416.00,1664416.00,1.749181e+06,4.610057,0.665004,4.695459,4.572559,4.672231,january,False
4,30,398527.0,UN2694,1169990555,2023-08-26,2023-08-24 14:59,2023-08-28,822706.00,411353.00,4.219834e+05,4.610057,0.665004,4.695459,4.572559,4.672231,august,False


### Create date features

In [39]:
data['booking_check_in'] = pd.to_datetime(data['booking_check_in'])
data['booking_check_out'] = pd.to_datetime(data['booking_check_out'])
data['created_at'] = pd.to_datetime(data['created_at'])

data['stay_duration'] = (data['booking_check_out'] - data['booking_check_in']).dt.days
data['booking_day_of_week'] = data['booking_check_in'].dt.dayofweek
data['booking_lead_time'] = (data['booking_check_in'] - data['created_at']).dt.days


### Calculate revenue metrics

In [40]:
data['total_earnings'] = data['earnings_in_idr']
data['price_fluctuation'] = data['earnings_per_day'] - data['average_baseline_price']

### Calculate review metrics

In [41]:
data['total_review_score'] = (data['rating'] + data['communication'] + data['cleanliness'] + data['accuracy']) / 4

### Display the updated dataframe

In [42]:
data.head()

,Unnamed: 0,room_id,unit_id,booking_id,booking_check_in,created_at,booking_check_out,earnings_in_idr,earnings_per_day,average_baseline_price,...,review_sentiment_score,communication,cleanliness,accuracy,stay_duration,booking_day_of_week,booking_lead_time,total_earnings,price_fluctuation,total_review_score
0,13,397667.0,UN2501,07-05-2023MartaRudy,2023-05-07,2023-05-07 08:58:00,2023-05-08,748425.01,748425.01,6.357513e+05,...,0.665004,4.695459,4.572559,4.672231,1,6,-1,748425.01,112673.752803,4.637576
1,25,397667.0,UN2501,1085232310,2023-09-02,2023-08-24 19:44:00,2023-09-06,2801652.00,700413.00,6.357513e+05,...,0.665004,4.695459,4.572559,4.672231,4,5,8,2801652.00,64661.742803,4.637576
2,26,394914.0,UN0375,1085251539,2023-09-11,2023-08-25 02:12:00,2023-09-13,2475126.00,1237563.00,1.325331e+06,...,0.665004,4.695459,4.572559,4.672231,2,0,16,2475126.00,-87767.568155,4.637576
3,29,387370.0,UN2760,1166180157,2024-01-20,2024-01-17 10:00:00,2024-01-21,1664416.00,1664416.00,1.749181e+06,...,0.665004,4.695459,4.572559,4.672231,1,5,2,1664416.00,-84764.976857,4.637576
4,30,398527.0,UN2694,1169990555,2023-08-26,2023-08-24 14:59:00,2023-08-28,822706.00,411353.00,4.219834e+05,...,0.665004,4.695459,4.572559,4.672231,2,5,1,822706.00,-10630.425230,4.637576


In [43]:
filepath=  'FE_RoomPrice.csv'
data.to_csv(filepath)